In [1]:
import pandas as pd
import requests
import json
import re

In [2]:
import plotly as py
import plotly.graph_objs as go
import plotly.express as px
py.offline.init_notebook_mode(connected = True)

In [3]:
commercial = pd.read_csv("상업업무용(매매)__실거래가_20200110101441.csv",skiprows=15,encoding="euc_kr")

In [4]:
# commercial["도로명"] = commercial["도로명"].map(lambda x: (re.findall(r'(\w+?)(\d+)', x))[0])
commercial["도로명"] = commercial["도로명"].str.replace("-","").str.split('(\d+)').map(lambda x: x[0])

In [5]:
commercial["면적당금액"] = commercial["거래금액(만원)"].str.replace(",","").astype(float)/commercial["전용/연면적(㎡)"].astype(float)
commercial["평당금액"] = commercial["면적당금액"]*.3025
commercial["주소"] = commercial["시군구"] + " " + commercial["도로명"]
adrs=commercial["주소"].unique()

In [6]:
commercial = commercial.pivot_table(index = ["주소","계약년월"],values=["평당금액"],aggfunc=["mean","count","std"])

In [61]:
len(adrs)

3431

In [62]:
geocodes = pd.DataFrame()
fail = []
n = 0
for adr in adrs:
        url = 'https://dapi.kakao.com/v2/local/search/keyword.json?query='+adr
        headers = {"Authorization": "KakaoAK 8809fcb48aa9900788adbd9f162c6b25"}
        result = json.loads(str(requests.get(url,headers=headers).text))
        #     return result
        try:
            match_first = result['documents'][0]
            apnd = pd.DataFrame()
            apnd["srch"] = [adr]
            apnd["y"] = [match_first['y']]
            apnd["x"] = [match_first['x']]
            apnd["address"] = [match_first["address_name"]]
            geocodes = geocodes.append(apnd)
        except IndexError:
            print(adr)
            fail.append(adr)

경기도 과천시 문원동 사기막
경기도 김포시 양촌읍 구래리 봉수대로
경기도 김포시 양촌읍 대포리 봉수대로
경기도 김포시 통진읍 서암리 검암
경기도 남양주시 별내면 청학리 청학로중앙길
경기도 남양주시 별내면 청학리 청학로응달길
경기도 남양주시 오남읍 양지리 양지로삭다니
경기도 남양주시 진건읍 용정리 독정로성지
경기도 남양주시 진접읍 금곡리 경복대로바람골길
경기도 남양주시 진접읍 연평리 금강로유연들
경기도 남양주시 화도읍 녹촌리 경춘보학
경기도 남양주시 화도읍 묵현리 먹갓로서길
경기도 남양주시 화도읍 창현리 경춘로창벌
경기도 시흥시 미산동 구수미
경기도 안산단원구 대부남동 한우물길
경기도 안산상록구 사동 당골
경기도 안산상록구 사동 네고지
경기도 안산상록구 이동 삼태기
경기도 안산상록구 이동 조구나리
경기도 안성시 일죽면 능국리 노동길
경기도 양주시 남면 신산리 개나리
경기도 양평군 단월면 향소리 윗소정길
경기도 양평군 양동면 매월리 광암길
경기도 양평군 지평면 월산리 월파길
경기도 여주시 금사면 상호리 금품
경기도 여주시 산북면 송현리 금품
경기도 여주시 산북면 후리 금품
경기도 의왕시 오전동 어내들
경기도 파주시 광탄면 마장리 소라울
경기도 파주시 월롱면 위전리 누현
경기도 파주시 적성면 구읍리 칠중
경기도 평택시 안중읍 안중리 안현로서
경기도 평택시 안중읍 현화리 안현로서
경기도 포천시 관인면 냉정리 찬우물
경기도 화성시 마도면 청원리 청원산단
경기도 화성시 마도면 청원리 청원산단로
경기도 화성시 봉담읍 덕리 덕머루
경기도 화성시 봉담읍 상리 진등
경기도 화성시 양감면 정문리 길목길
경기도 화성시 우정읍 매향리 매향요골길
경기도 화성시 우정읍 화산리 장포한말길
경기도 화성시 정남면 관항리 만은동
경기도 화성시 향남읍 하길리 발안양감로


In [64]:
# geocodes.to_csv("geocodes_kr_dong.csv",index=False,encoding="euc_kr")
geocodes.to_csv("geocodes_kr_rd(경기).csv",index=False,encoding="euc_kr")

In [7]:
geocodes = pd.read_csv("geocodes_kr_rd(경기).csv",encoding="euc_kr")

In [8]:
commercial.reset_index(inplace = True)

In [9]:
commercial.rename(columns={"주소":"srch"},inplace = True)

In [10]:
df = pd.merge(geocodes,commercial,on="srch")

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\reshape\merge.py:522: UserWarning:

merging between different levels can give an unintended result (1 levels on the left, 2 on the right)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:3812: PerformanceWarning:

dropping on a non-lexsorted multi-index without a level parameter may impact performance.



In [12]:
df.columns = ["srch","y","x","address","계약년월","mean","count","std"]
df.sort_values("계약년월",inplace= True)

In [13]:
df["x"] = df["x"].astype(float)
df["y"] = df["y"].astype(float)

In [30]:
df.head()

,srch,y,x,address,계약년월,mean,count,std
0,경기도 가평군 가평읍 금대리,37.770261,127.517540,경기 가평군 가평읍 금대리,201905,22.482099,1,NaN
1,경기도 가평군 가평읍 금대리,37.770261,127.517540,경기 가평군 가평읍 금대리,201906,33.031609,2,9.014934
2,경기도 가평군 가평읍 금대리,37.770261,127.517540,경기 가평군 가평읍 금대리,201911,30.207052,2,10.185328
3,경기도 가평군 가평읍 달전리,37.815752,127.527947,경기 가평군 가평읍 달전리 30,201912,104.114418,1,NaN
4,경기도 가평군 가평읍 대곡리,37.817992,127.518641,경기 가평군 가평읍 대곡리 683,201902,115.246225,1,NaN


In [14]:
px.set_mapbox_access_token("pk.eyJ1IjoidGl2bWU3IiwiYSI6ImNrMWEwZDVtNDI4Zm4zYm1vY3o3Z25zejEifQ._yTPkj3nXTzor72zIevLCQ")
fig = px.scatter_mapbox(df, lat="y", lon="x", size="mean", color = "mean", hover_name="srch",
                      color_continuous_scale=px.colors.diverging.Portland, size_max=30, zoom=3,animation_frame="계약년월")
fig.update_layout(autosize=True,width=1500,height=750)
    #                           ,margin=go.layout.Margin(l=50,r=50,b=100,t=100,pad=4))
py.offline.plot(fig)

'temp-plot.html'